In [1]:
import sys
import os
import signal

from isaacsim import SimulationApp

CONFIG = {
  'headless': False,
  'display_options': 3286
}

# Example for creating a RTX lidar sensor and publishing PointCloud2 data
simulation_app = SimulationApp(CONFIG)
simulation_app.update()

spinning = True
def sigint_handler(sig, frame):
  global spinning
  spinning = False
signal.signal(signal.SIGINT, sigint_handler)

kernel = get_ipython().kernel
kernel.pre_handler_hook = lambda: None
kernel.post_handler_hook = lambda: None

def spin(update_function, *args, **kwargs):
  global spinning
  print('gui can be used now. interrupt kernel to go back to notebook')
  while spinning:
    update_function(*args, **kwargs)
  spinning = True

def resume_gui():
  spin(lambda: simulation_app.update())
  
if CONFIG['headless']:
  del spin
  del resume_gui
  

print('workdir:', os.getcwd())


Starting kit application with the following args:  ['/f/ov/pkg/isaac-sim-4.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/f/ov/pkg/isaac-sim-4.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/f/ov/pkg/isaac-sim-4.1.0/kit', '--/persistent/app/viewport/displayOptions=3286', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/exts', '--ext-folder', '/f/ov/pkg/isaac-sim-4.1.0/apps', '--/physics/cudaDevice=0', '--portable']
Passing the following args to the base kit application:  ['-f', '/home/linh/.local/share/jupyter/runtime/kernel-7c920987-5d07-4710-a838-51cec276805f.json']
[Warning] [omni.isaac.kit.simulation_app] Non-headless mode not supported with jupyt

Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 4060" (8 GiB, sm_89, mempool enabled)
   Kernel cache:
     /home/linh/.cache/warp/1.2.1
] [ext: omni.kit.widget.text_editor-1.0.2] startup
[2.907s] [ext: omni.kit.viewport.window-106.0.8] startup
[2.929s] [ext: omni.kit.window.property-1.11.1] startup
[2.932s] [ext: omni.usdphysics-106.0.20] startup
[2.936s] [ext: omni.graph.tools-1.78.0] startup
[2.960s] [ext: omni.kit.viewport.utility-1.0.17] startup
[2.961s] [ext: omni.kit.property.usd-3.21.28] startup
[2.971s] [ext: omni.graph-1.135.0] startup
[3.098s] [ext: omni.graph.image.core-0.3.2] startup
[3.103s] [ext: omni.graph.ui-1.70.0] startup
[3.121s] [ext: omni.graph.action_core-1.1.4] startup
[3.131s] [ext: omni.kit.window.cursor-1.1.2] startup
[3.133s] [ext: omni.graph.image.nodes-1.0.2] startup
[3.135s] [ext: omni.graph.action_nodes-1.23.0] startup
[3.146s] [ext: omni.kit.viewport.menubar.core-106.

In [2]:
# App/Create Custom Event
import carb.events
import omni.kit.app

# Event is unique integer id. Create it from string by hashing, using helper function.
# [ext name].[event name] is a recommended naming convention:
MY_CUSTOM_EVENT = carb.events.type_from_string("omni.my.extension.MY_CUSTOM_EVENT")

# App provides common event bus. It is event queue which is popped every update (frame).


class EventHandler:
    def __init__(self):
        bus = omni.kit.app.get_app().get_message_bus_event_stream()        
        self.sub1 = bus.create_subscription_to_push_by_type(MY_CUSTOM_EVENT, self.on_push_event)
        self.sub2 = bus.create_subscription_to_pop_by_type(MY_CUSTOM_EVENT, self.on_pop_event)
        
    def on_push_event(self, e):
        print('pushed:', e.type, e.type == MY_CUSTOM_EVENT)

    def on_pop_event(self, e):
        print('popped:', e.type, e.type == MY_CUSTOM_EVENT)


event_handler = EventHandler()

# Push event the bus with custom payload
msgbus = omni.kit.app.get_app().get_message_bus_event_stream()  
msgbus.push(MY_CUSTOM_EVENT, payload={"data": 2, "x": "y"})

pushed: 7675526397818946258 True


In [3]:
msgbus.push(MY_CUSTOM_EVENT, payload={"data": 2, "x": "y"})


pushed: 7675526397818946258 True


In [5]:
msgbus.event_count

2

In [6]:
simulation_app.update()

popped: 7675526397818946258 True
popped: 7675526397818946258 True


In [7]:
msgbus.event_count

0

In [13]:
import numpy as np

for _ in range(10):
    payload = np.random.rand(1000*1000*250) # about 1GB a message
    msgbus.push(MY_CUSTOM_EVENT, payload={"data": payload})
    simulation_app.update()

pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True
pushed: 7675526397818946258 True
popped: 7675526397818946258 True


In [14]:
msgbus.event_count

0

In [15]:
resume_gui()

gui can be used now. interrupt kernel to go back to notebook
